# **Set according to environment (e.g. local, Google Colab...)**

In [1]:
project_folder = ''

# **Body**

In [2]:
from custom_libraries.miscellaneous import *
from custom_libraries.image_dataset import *
import numpy as np

In [3]:
from keras import backend
import math

def init_weights(masker):

    density = np.count_nonzero(masker == 1) / masker.size
    stddev = math.sqrt(2/(masker.shape[0]*density))
    weights = np.random.normal(size=masker.shape, loc = 0., scale=stddev)
    weights[np.where(masker == 0)] = 0
    weights = np.reshape(weights, [-1])
    weights = weights[np.where(weights != 0)]
    return weights

class treeLayer(tf.keras.layers.Layer):

    def __init__(self, Input_size=3072, Activation="relu"):
        super(treeLayer, self).__init__()
        self.bias = None
        self.summer = None
        self.kernel = None
        self.Input_size = Input_size
        self.Activation = Activation

    def build(self, input_shape):

        self.summer = np.zeros((self.Input_size, self.Input_size // 2))
        for i in range(self.Input_size):
            self.summer[i, i // 2] = 1

        initializer = init_weights(self.summer)
        initializer = tf.keras.initializers.Constant(initializer)

        self.kernel = self.add_weight(shape=(1, self.Input_size),
                                      initializer=initializer,
                                      trainable=True, dtype=tf.float32)

        self.summer = tf.convert_to_tensor(self.summer, dtype=tf.float32)
        self.bias = self.add_weight(shape=(self.Input_size//2,),
                                    initializer=tf.keras.initializers.Zeros,
                                    trainable=True)

    def call(self, inputs):
        #print(inputs.shape, self.kernel.shape)
        x = tf.math.multiply(inputs, self.kernel)
        x = tf.matmul(x, self.summer)
        #masked_weights = tf.multiply(self.kernel, self.summer)
        #x = tf.matmul(inputs, masked_weights)
        x = tf.nn.bias_add(x, self.bias)
        x = tf.nn.leaky_relu(x, alpha = .01)
        return x

def create_model(input_size, num_trees=1):
    model = tf.keras.Sequential()
    while input_size > num_trees:
        model.add(treeLayer(input_size))
        input_size = input_size // 2
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid', use_bias=True))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, epsilon=1e-08),
                  loss=tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE),
                  metrics=['binary_crossentropy', 'acc'])

    return model


In [4]:
import gc

# Initialize settings
bs = 256
trials = 10
epochs = 2000
trees_set = [1]

# Load class-dataset list
# classes = np.load(project_folder + 'results/classes.npy', allow_pickle=True)

classes = [[3, 5, 'mnist'],
           [0, 6, 'fmnist'],
           [2, 6, 'kmnist'],
           [14, 17, 'emnist']]

callback = tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=60)

#history = np.load(project_folder + 'results/fcnn_history.npy', allow_pickle=True)
history = np.zeros((len(classes), trials, len(trees_set), 2))

# For each dataset enumerated from classes list
for j, (t1, t2, ds) in enumerate(classes):

    print(f"Dataset: {ds} / Pair: {t1}-{t2}")

    test_ds = ImageDataset(ds, 'test', data_dir=None)
    train_ds = ImageDataset(ds, 'train', data_dir=None)

    for x in [train_ds, test_ds]:
        x.filter(t1, t2, overwrite=True)
        x.shuffle()
        x.normalize()
        if x.images.shape[1:3] == (28, 28):
            x.pad()
        x.vectorize(True)

    for k, trees in enumerate(trees_set):

        print(f"{trees}-FCNN")
        X_train, y_train, X_valid, y_valid = train_ds.subset(shard=True, shard_number=trials, validation=True,
                                                             validation_size=len(test_ds.images))
        X_test, y_test = test_ds.subset()
        test_set = tf.data.Dataset.from_tensor_slices((X_test, y_test)).map(
            lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)

        #if history[j, -1, k, 0] != 0:
        #  continue

        for i in range(trials):
            print(f"Trial {i + 1}")

            #with tf.device('/device:GPU:0'):

            model = create_model(input_size=X_train[i].shape[1] * trees, num_trees=trees)

            train_set = tf.data.Dataset.from_tensor_slices((X_train[i], y_train[i])).map(
                lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)
            valid_set = tf.data.Dataset.from_tensor_slices((X_valid[i], y_valid[i])).map(
                lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)

            fit_history = model.fit(x=train_set, batch_size=bs, epochs=epochs,
                                    validation_data=valid_set, validation_batch_size=bs,
                                    callbacks=[callback], verbose=0)
            print_fit_history(fit_history, epochs)

            evaluate_history = model.evaluate(x=test_set, batch_size=bs, verbose=0)
            print_evaluate_history(evaluate_history)

            history[j, i, k] = evaluate_history[1:]

            np.save(project_folder + 'results/ktree_history.npy', history,
                    allow_pickle=True)

            del model, train_set, valid_set
            gc.collect()

Dataset: mnist / Pair: 3-5
1-FCNN
Trial 1
Epochs: 436/2000 - Train BCE: 0.2134, accuracy: 93.47% - Validation BCE: 0.2852, accuracy: 90.05%
Test BCE: 0.2618, accuracy: 90.59%
Trial 2
Epochs: 605/2000 - Train BCE: 0.1789, accuracy: 93.78% - Validation BCE: 0.2372, accuracy: 92.15%
Test BCE: 0.3069, accuracy: 89.06%
Trial 3
Epochs: 748/2000 - Train BCE: 0.1522, accuracy: 95.44% - Validation BCE: 0.1852, accuracy: 93.16%
Test BCE: 0.2401, accuracy: 92.32%
Trial 4
Epochs: 77/2000 - Train BCE: 0.6842, accuracy: 56.27% - Validation BCE: 0.6912, accuracy: 53.68%
Test BCE: 0.6926, accuracy: 53.1%
Trial 5


KeyboardInterrupt: 

In [ ]:
history = np.load(project_folder + 'results/ktree_history.npy', allow_pickle=True)
print("RESULTS:")
for j, (t1, t2, ds) in enumerate(classes):
    print(f"Dataset: {ds} / Pair: {t1}-{t2}")
    for k, trees in enumerate(trees_set):
        print(f"{trees}-tree")
        print(f"Accuracy: mean = {round(np.mean(history[j,:,k,1]), 4)}, standard deviation = {round(np.std(history[j,:,k,1]), 4)}")